In [ ]:
#! pip install PyPDF2

  Obtaining dependency information for PyPDF2 from https://files.pythonhosted.org/packages/8e/5e/c86a5643653825d3c913719e788e41386bee415c2b87b4f955432f2de6b2/pypdf2-3.0.1-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/232.6 kB ? eta -:--:--
   - -------------------------------------- 10.2/232.6 kB ? eta -:--:--
   ---------- ---------------------------- 61.4/232.6 kB 812.7 kB/s eta 0:00:01
   ---------------------------------------- 232.6/232.6 kB 2.4 MB/s eta 0:00:00



[notice] A new release of pip is available: 23.2.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import pandas as pd
from elasticsearch import Elasticsearch
from elasticsearch.client import MlClient

C:\Users\baek2\AppData\Local\Temp\ipykernel_41108\804421021.py:3: DeprecationWarning: Importing from the 'elasticsearch.client' module is deprecated. Instead use 'elasticsearch' module for importing the client.
  from elasticsearch.client import MlClient


In [ ]:
# ES 클라이언트 정의
import os
from dotenv import load_dotenv
from elasticsearch import Elasticsearch

load_dotenv()

url = os.getenv("ELASTIC_CLOUD_URL")
api_id = os.getenv("ELASTIC_API_ID")
api_key = os.getenv("ELASTIC_API_KEY")
es_model_id = os.getenv("ELASTIC_MODEL_ID")


client = Elasticsearch(
    url,
    api_key=(api_id, api_key)
)

print(client.info())

{'name': 'instance-0000000000', 'cluster_name': 'b675458383584374af04fcee51788385', 'cluster_uuid': 'tEQ5uK6mSq6qmqujcMeoLg', 'version': {'number': '9.2.1', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '4ad0ef0e98a2e72fafbd79a19fa5cae2f026117d', 'build_date': '2025-11-06T22:07:39.673130621Z', 'build_snapshot': False, 'lucene_version': '10.3.1', 'minimum_wire_compatibility_version': '8.19.0', 'minimum_index_compatibility_version': '8.0.0'}, 'tagline': 'You Know, for Search'}


In [44]:
import os
import glob
from typing import List, Dict

import pandas as pd
from PyPDF2 import PdfReader

# 1. PDF > CSV

In [ ]:
def extract_text_from_pdf(pdf_path: str) -> str:
    """
    단일 PDF 파일에서 모든 페이지 텍스트를 추출하는 함수.
    """
    reader = PdfReader(pdf_path)
    texts = []

    for page in reader.pages:
        page_text = page.extract_text()
        if page_text:
            texts.append(page_text)

    return "\n".join(texts)


In [8]:

def chunk_text(
    text: str,
    max_chars: int = 1000,
    overlap: int = 200
) -> List[str]:
    """
    긴 텍스트를 max_chars 기준으로 잘라 chunk 리스트를 반환.
    overlap만큼 앞 chunk와 겹치게 슬라이딩 윈도우 형태로 자름.
    """
    text = " ".join(text.split())
    if not text:
        return []

    chunks = []
    start = 0
    text_length = len(text)

    while start < text_length:
        end = start + max_chars
        chunk = text[start:end]
        chunks.append(chunk)

        # 다음 chunk의 시작 인덱스 (overlap 적용)
        start = end - overlap
        if start < 0:
            start = 0
        if start >= text_length:
            break

    return chunks


In [9]:
def build_pdf_chunk_dataframe(
    folder_path: str,
    max_chars: int = 1000,
    overlap: int = 200
) -> pd.DataFrame:
    """
    주어진 폴더에서 모든 PDF를 읽고
    파일별로 텍스트 추출 → chunking → DataFrame 생성.
    컬럼: filename, chunk_seq, chunk_text
    """
    pdf_files = glob.glob(os.path.join(folder_path, "*.pdf"))

    records: List[Dict] = []

    for pdf_path in pdf_files:
        filename = os.path.basename(pdf_path)
        print(f"Processing: {filename}")

        full_text = extract_text_from_pdf(pdf_path)
        chunks = chunk_text(full_text, max_chars=max_chars, overlap=overlap)

        for i, chunk in enumerate(chunks, start=1):
            records.append(
                {
                    "filename": filename,
                    "chunk_seq": i,
                    "chunk_text": chunk,
                }
            )

    df = pd.DataFrame(records, columns=["filename", "chunk_seq", "chunk_text"])
    return df


In [11]:
folder_path = "D:/workspace/대학원/25년도2학기/정보검색프로젝트/색인데이터"

max_chars = 1000
overlap = 200

df = build_pdf_chunk_dataframe(
    folder_path=folder_path,
    max_chars=max_chars,
    overlap=overlap
    )

print(df.head())

Processing: 3주차 검색개요 - 색인과 검색랭킹과 평가(part2).pdf
Processing: 4주차 ElasticSearch.pdf
Processing: 5주차 LLM 이해와 PromptEngineering 9월 30일.pdf
Processing: 7주차 생성형AI 검색 기초 - 벡터검색과 RAG_1014.pdf
Processing: LangChain 기초_별첨.pdf
Processing: 제10강 검색에이전트와 LangChain LangGraph.pdf
Processing: 제11강 LangChain LangGraph를 이용한 AgenticRAG 개발.pdf
Processing: 제12강 Multi-Hop .pdf
                             filename  chunk_seq  \
0  3주차 검색개요 - 색인과 검색랭킹과 평가(part2).pdf          1   
1  3주차 검색개요 - 색인과 검색랭킹과 평가(part2).pdf          2   
2  3주차 검색개요 - 색인과 검색랭킹과 평가(part2).pdf          3   
3  3주차 검색개요 - 색인과 검색랭킹과 평가(part2).pdf          4   
4  3주차 검색개요 - 색인과 검색랭킹과 평가(part2).pdf          5   

                                          chunk_text  
0  빅데이터 정보검색 제3강정보검색 개요: 검색랭킹모델과평가 황영숙 (youngsook...  
1  VectorSpaceModel ): 문서와 질의를 고차원 벡터로 표현하고 , 두 벡...  
2   않음 •대다수 사용자는 불리언 질의를 작성하는 데 어려움을 느끼거나 , 작성하는 ...  
3  ccard (A,B) = 0if A ∩ B = 0 •한계점 •단어 빈도(term f...  
4  00 4 sunday 1,000 3 fly 10,000 2 under 100,000...  


In [14]:
df.loc[0]['chunk_text']

'빅데이터 정보검색 제3강정보검색 개요: 검색랭킹모델과평가 황영숙 (youngsook_hwang@korea.ac.kr) 정보검색개요 CollectionUser task Info need Query ResultsSearch engine Query refinement 정보 검색 개요 •정보 검색의 핵심 구성요소 •색인 (Indexing) : 수많은 문서를 효율적으로 검색할 수 있도록 구조화 하는 과정 •전처리 •토큰화 (Tokenization): 문서를 단어 단위로 쪼개는 과정 •불용어 제거(Stop Word Removal): ‘ 은’, ‘는‘ , ‘이‘, ‘가‘, ‘the’, ‘a’ 등 의미없는 단어 제거 •어간추출 (Stemming)/ 표제어 추출(Lemmatization): 단어의 원형 복원(예: runs →run, running→run, 먹었다→ 먹다) •키워드 역색인 (Inverted Index): 각 토큰이 어느 문서에서 출현했는지 기록하는 자료구조 •질의(Query): 사용자가 찾는 정보의 표현 •키워드 질의 •자연어 질의: ‘인공지능의 최신 연구동향에 대한 논문을 찾아줘 ‘와 같은 자연어 질문 •랭킹(Ranking): 문서와 질의간 유사도를 계산하여 질의와 관련된 문서를 중요도 순으로 나열하는 과정 •TF-IDF: 토큰 빈도(문서 내 특정 단어의 출현 빈도)와 역문서 빈도(전체 문서 중 특정 단어가 포함된 문서의 수)를 사 용하여 문서 내 용어의 중요도를 평가하는 가장 기본적인 방법 •BM25: TF -IDF를 개선한 확률적 모델로 , 많은 검색 엔진에서 기본 랭킹 알고리즘으로 사용 •벡터공간 모델(VectorSpaceModel ): 문서와 질의를 고차원 벡터로 표현하고 , 두 벡터 간의 코사인 유사도 (Cosine Similarity) 를 측정하여 랭킹. 최근 워드임베딩 , BERT 와 같은 딥러닝 기반의 언어 모델을 많이 사용 정보 검색 개요 •정보검색의 성능을 측정하는 주요 지표들 •정확성 (Relevance) : 검색된 문서가 사용자의 질의와

In [16]:
df.loc[1]['chunk_text']

'VectorSpaceModel ): 문서와 질의를 고차원 벡터로 표현하고 , 두 벡터 간의 코사인 유사도 (Cosine Similarity) 를 측정하여 랭킹. 최근 워드임베딩 , BERT 와 같은 딥러닝 기반의 언어 모델을 많이 사용 정보 검색 개요 •정보검색의 성능을 측정하는 주요 지표들 •정확성 (Relevance) : 검색된 문서가 사용자의 질의와 얼마나 관련 있는지를 판단하는 주관적인 기준으로 전문가 그룹이 판단 •정밀도 (Precision): 검색된 문서 중 실제로 관련 있는 문서의 비율 •재현율 (Recall): 전체 관련 문서 중 실제로 검색된 문서의 비율 •F1-점수(F1-score): 정밀도와 재현율의 조화 평균 •그외 지표 •MAP(Mean Average Precision) : 질의에 대한 평균 정밀도를 여러 질의에 대해 평균낸 값 •nDCG (normalized Discounted Cumulative Gain) : 랭킹 순서를 고려하여 상위 랭크의 중요도를 높게 평가하는 지표 랭킹 •Ranked Retrieval •Scoring Documents •Tf-Idfweighting •Vector Space Model •BM25, BM25F 랭킹 기반 검색(Ranked Retrieval) •Boolean Query 의 장점과 한계 •블리언 질의: 문서가 질의와 일치하거나 , 그렇지 않거나 둘 중 하나 •장점 •자신의 요구사항과 문서 집합을 정확하게 이해하는 전문가 에게 유용 •애플리케이션이 수천 개의 결과를 쉽게 처리할 수 있어 응용 프로그램 에 적합 •한계 •대부분의 일반 사용자에게는 적합하지 않음 •대다수 사용자는 불리언 질의를 작성하는 데 어려움을 느끼거나 , 작성하는 것을 번거로워함 •대부분의 사용자는 수천 개의 결과 목록을 일일이 확인하는 것을 원하지 않음 •결과 수의 불균형 : 종종 결과의 수가 너무 적거나 (0개) 혹은 너무 많은(수천 개) 극단적 상황 초래 →적절한 수의 결과를 얻기 위해서는 상당한 수준의 질의 작성 기술이 필요 

In [17]:
df.to_csv("pdf_chunks.csv", index=False, encoding="utf-8-sig")

In [4]:

class_info = pd.read_csv('./pdf_chunks.csv', encoding='utf-8-sig')
class_info.loc[:1]

,filename,chunk_seq,chunk_text
0,3주차 검색개요 - 색인과 검색랭킹과 평가(part2).pdf,1,빅데이터 정보검색 제3강정보검색 개요: 검색랭킹모델과평가 황영숙 (youngsook...
1,3주차 검색개요 - 색인과 검색랭킹과 평가(part2).pdf,2,"VectorSpaceModel ): 문서와 질의를 고차원 벡터로 표현하고 , 두 벡..."


In [28]:
class_info

,filename,chunk_seq,chunk_text
0,3주차 검색개요 - 색인과 검색랭킹과 평가(part2).pdf,1,빅데이터 정보검색 제3강정보검색 개요: 검색랭킹모델과평가 황영숙 (youngsook...
1,3주차 검색개요 - 색인과 검색랭킹과 평가(part2).pdf,2,"VectorSpaceModel ): 문서와 질의를 고차원 벡터로 표현하고 , 두 벡..."
2,3주차 검색개요 - 색인과 검색랭킹과 평가(part2).pdf,3,"않음 •대다수 사용자는 불리언 질의를 작성하는 데 어려움을 느끼거나 , 작성하는 ..."
3,3주차 검색개요 - 색인과 검색랭킹과 평가(part2).pdf,4,"ccard (A,B) = 0if A ∩ B = 0 •한계점 •단어 빈도(term f..."
4,3주차 검색개요 - 색인과 검색랭킹과 평가(part2).pdf,5,"00 4 sunday 1,000 3 fly 10,000 2 under 100,000..."
...,...,...,...
219,제12강 Multi-Hop .pdf,16,(Decomposed Retrieval ) •Self-Correction: 검색된 ...
220,제12강 Multi-Hop .pdf,17,"파악하고 , 이를 포함하는 다음 검색 쿼리를 생성해야 함(예: ""찰스 디킨스의 대..."
221,제12강 Multi-Hop .pdf,18,ation) 단계의 주요 이슈 •복잡한 논리적 통합 (Complex Logical ...
222,제12강 Multi-Hop .pdf,19,"-to-Most Prompting •Denny Zhou, et al ., “Leas..."


# 2. ES 색인 시작

In [5]:
index_name = "class-info"

In [ ]:
# 2.1 임베딩을 위한 ingest pipeline 추가 

client.ingest.put_pipeline(
    id="pipeline",
    processors=[
        {
            "inference": {
                "model_id": es_model_id,
                "field_map": {"chunk_text": "text_field"},  # field to embed: information
                "target_field": "chunk_embedding",  # embedded field: information_embedding
            }
        }
    ],
)

ObjectApiResponse({'acknowledged': True})

In [ ]:
# 2.2 색인을 위한 analyzer, mapping 구성, pipeline 연결 

index_body = {
    "settings": {
        "index.mapping.exclude_source_vectors": False, 
        "analysis": {
            "tokenizer": {
                "nori_tokenizer_custom": {
                    "type": "nori_tokenizer",
                    "decompound_mode": "mixed"
                }
            },
            "analyzer": {
                "korean_nori": {
                    "type": "custom",
                    "tokenizer": "nori_tokenizer_custom",
                    "filter": [
                        "lowercase"
                    ]
                }
            }
        },
        "index": {
        "number_of_replicas": "1",
        "number_of_shards": "1",
        "default_pipeline": "pipeline",
        }
    },
    "mappings": {
        "properties": {
            "filename": {
                "type": "keyword"
            },
            "chunk_seq": {
                "type": "integer"
            },
            "chunk_text": {
                "type": "text",
                "analyzer": "korean_nori"
            },
            "chunk_embedding.predicted_value": {
                "type": "dense_vector",
                "dims": 768,
                "index": True,
                "similarity": "cosine"
            }
        }
    }
}

In [ ]:
# if client.indices.exists(index=index_name):
#     client.indices.delete(index=index_name)
#     print(f"Deleted existing index: {index_name}")

Deleted existing index: class-info


In [ ]:
# 2.3 인덱스 생성
response = client.indices.create(
    index=index_name,
    body=index_body
)

In [ ]:
# 2.4 데이터 색인

indexed_results = []

for index, row in class_info.iterrows():
    doc_source = {
        "filename": row['filename'] if pd.notna(row['filename']) else None,
        "chunk_seq": row['chunk_seq'] if pd.notna(row['chunk_seq']) else None,
        "chunk_text": row['chunk_text'] if pd.notna(row['chunk_text']) else None
    }
    
    try:
        response = client.index(index="class-info", document=doc_source, id=str(index))
        indexed_results.append({'index': index, 'status': 'success', 'response': response})
        if index % 30 == 0:
            print(f"Successfully indexed document with index {index}")
    except Exception as e:
        indexed_results.append({'index': index, 'status': 'failed', 'error': str(e)})
        print(f"Failed to index document with index {index}: {e}")


Successfully indexed document with index 0
Successfully indexed document with index 30
Successfully indexed document with index 60
Successfully indexed document with index 90
Successfully indexed document with index 120
Successfully indexed document with index 150
Successfully indexed document with index 180
Successfully indexed document with index 210
